This notebook generates dataset according to CrossDock 100k subset
- The goal is to use full protein from original CrossDock dataset instead of the version with 12A cutoff.
- pockets not from PDBBind set are removed 

You need to have CrossDock subset and full set downloaded to run this script.

In [22]:
import os
import torch
from tqdm import tqdm
from collections import defaultdict 
import sys

sys.path.append('../')

In [23]:
from glob import glob

pockets = glob('../dataset/crossdock/*_rec.pdb')
len(pockets)

0

In [3]:
lig = glob('../dataset/crossdock/*_lig.pdb')
len(lig)

21231

In [4]:
pocket_id = [i.split('/')[-1].split('_')[0] for i in pockets]
lig_id = [i.split('/')[-1].split('_')[0] for i in lig]

intersect = set(pocket_id).intersection(set(lig_id))

In [5]:
len(intersect)

19695

In [6]:
from src.tacogfn.utils import molecules

In [7]:
# Load CrossDock dataset split pocket by sequence file
split_file = torch.load('../dataset/split_by_name.pt')
merged_split_file = (split_file['train'] + split_file['test'])

In [8]:
# Convert path to pocket id and ligand id
get_pocket_id_and_ligand_id = lambda path: (path.split('/')[1].split('_rec')[0], path.split('/')[1].split('rec_')[1].split('_lig')[0])

In [9]:
# Convert path into pocket ligand pairs
pairs = [get_pocket_id_and_ligand_id(path) for path, _ in merged_split_file]

In [10]:
# map pocket to list of ligands
pocket_to_ligands = defaultdict(list)

for pocket, ligand in pairs:
    pocket_to_ligands[pocket].append(ligand)

In [11]:
# map pocket to the native ligand, if native ligand is not in the list of ligands, then use the first ligand
pocket_to_best_ligand = {}
for pocket, ligand_list in pocket_to_ligands.items():
    pocket_id = pocket.split('_')[0]
    
    # default to first ligand
    pocket_to_best_ligand[pocket] = ligand_list[0]
    
    for ligand in ligand_list:
        if ligand.split('_')[0] == pocket_id:
            pocket_to_best_ligand[pocket] = ligand

In [12]:
# Convert the pocket id to pdb path in CrossDock
# Create a map of pocket id to pdb path
pocket_id_to_pdb_path = {}

for pocket_rec_id, _ in pocket_to_best_ligand.items():
    path = f'../dataset/crossdock/{pocket_rec_id}_rec.pdb'
    
    if os.path.exists(path):
        pocket_id_to_pdb_path[pocket_rec_id] = path
    else:
        pocket_id_to_pdb_path[pocket_rec_id] = None  

In [13]:
num_pockets_not_in_pdbbind = len({k: v for k, v in pocket_id_to_pdb_path.items() if v is None})
print(f'Number of pockets not in pdbbind: {num_pockets_not_in_pdbbind}')

Number of pockets not in pdbbind: 831


In [14]:
# Convert the pocket_to_best_ligand in paths
protein_path_to_ligand_path = {}

for pocket_path, ligand_path in merged_split_file:
    pocket_id, ligand_id = get_pocket_id_and_ligand_id(pocket_path)
    
    best_ligand_id = pocket_to_best_ligand[pocket_id]
    protein_path = pocket_id_to_pdb_path[pocket_id]
    if best_ligand_id == ligand_id and protein_path is not None:
        protein_path_to_ligand_path[protein_path] = os.path.join('../dataset/crossdocked_pocket10/', ligand_path)    

In [21]:
# move the files into a processed folder 
# make sure we align the pocket and the whole protein - and then we can adjust the ligand accordingly

processed_protein_id_to_ligand_id = {}
failed_dict = {}

for protein_path, ligand_path in tqdm(protein_path_to_ligand_path.items()):
    
    pocket_path = ligand_path.replace('.sdf', '_pocket10.pdb')
    assert os.path.exists(pocket_path)
    
    protein_rec_id = os.path.splitext(os.path.basename(protein_path))[0]
    protein_rec_filename = f'{protein_rec_id}.pdb'
    
    ligand_sdf_id = ligand_path.split('/')[-1].split('lig')[0] + 'lig' 
    
    processed_protein_path = os.path.join('../dataset/processed/proteins/', protein_rec_filename)
    processed_ligand_path = os.path.join('../dataset/processed/ligands/', f'{ligand_sdf_id}.sdf')
    
    
    os.system(f'cp {protein_path} {processed_protein_path}')
    # try:
    try:
        rot_mat, original_center, new_center = molecules.get_rototranslation_alignment(
            protein_path, 
            pocket_path,
        )

        molecules.transform_sdf(
            input_file=ligand_path, 
            output_file=processed_ligand_path, 
            rotation_matrix=rot_mat, 
            original_center=original_center, 
            new_center=new_center
        )
    except:
        os.system(f'cp {ligand_path} {processed_ligand_path}')
        print(processed_protein_path)
        print(processed_ligand_path)
        break
    
    processed_protein_id_to_ligand_id[protein_rec_id] = ligand_sdf_id
    # except:
    #     failed_dict[protein_rec_id] = ligand_sdf_id

  0%|          | 2/14476 [00:00<11:39, 20.69it/s]

../dataset/processed/proteins/5acc_A_rec.pdb
../dataset/processed/ligands/5acc_A_rec_5acc_ke9_lig.sdf


In [18]:
# Get the train and test split for CrossDock pockets
train_pockets = set([get_pocket_id_and_ligand_id(path)[0] for path, _ in split_file['train']])
test_pockets = set([get_pocket_id_and_ligand_id(path)[0] for path, _ in split_file['test']])

In [19]:
# Create a new split file for CrossDock 

new_split_file = {'train': {}, 'test': {}}

for pocket_id, ligand_id in processed_protein_id_to_ligand_id.items():
    pocket_id = pocket_id.split('_rec')[0]
    if pocket_id in train_pockets:
        new_split_file['train'][pocket_id] = ligand_id
    elif pocket_id in test_pockets:
        new_split_file['test'][pocket_id] = ligand_id
    else:
        raise Exception('Pocket not in train or test split')


In [20]:
torch.save(new_split_file, '../dataset/processed/processed_split_file.pt')